# ओपनबीएनबी एमसीपी सर्व्हर एकत्रीकरणासह सेमॅंटिक कर्नल

हे नोटबुक सेमॅंटिक कर्नलचा वापर करून वास्तविक ओपनबीएनबी एमसीपी सर्व्हरसोबत कसे काम करावे हे दाखवते, ज्यामुळे MCPStdioPlugin चा वापर करून खऱ्या Airbnb निवासस्थानांचा शोध घेता येतो. LLM प्रवेशासाठी, हे Azure AI Foundry चा वापर करते. तुमचे पर्यावरणीय चल सेट करण्यासाठी, तुम्ही [सेटअप धडा](/00-course-setup/README.md) अनुसरू शकता.


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP प्लगइन कनेक्शन तयार करणे

आपण [OpenBnB MCP सर्व्हर](https://github.com/openbnb-org/mcp-server-airbnb) शी MCPStdioPlugin वापरून कनेक्ट होऊ. हा सर्व्हर @openbnb/mcp-server-airbnb पॅकेजद्वारे Airbnb शोध कार्यक्षमता प्रदान करतो.


## क्लायंट तयार करणे

या नमुन्यात, आपण LLM प्रवेशासाठी Azure AI Foundry वापरणार आहोत. आपले पर्यावरणीय चल (environment variables) योग्यरित्या सेट केलेले असल्याची खात्री करा.


## पर्यावरण कॉन्फिगरेशन

Azure OpenAI सेटिंग्ज कॉन्फिगर करा. खालील पर्यावरणीय व्हेरिएबल्स सेट केल्या आहेत याची खात्री करा:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP एकत्रीकरण समजून घेणे

हा नोटबुक **खऱ्या OpenBnB MCP सर्व्हरला** जोडतो, जो वास्तविक Airbnb शोध कार्यक्षमता प्रदान करतो.

### हे कसे कार्य करते:

1. **MCPStdioPlugin**: MCP सर्व्हरसोबत मानक इनपुट/आउटपुट संवादाचा वापर करते
2. **खरा NPM पॅकेज**: `@openbnb/mcp-server-airbnb` npx च्या माध्यमातून डाउनलोड आणि चालवते
3. **थेट डेटा**: त्यांच्या API कडून वास्तविक Airbnb मालमत्तेचा डेटा परत करते
4. **फंक्शन शोध**: एजंट MCP सर्व्हरमधून उपलब्ध फंक्शन्स आपोआप शोधतो

### उपलब्ध फंक्शन्स:

OpenBnB MCP सर्व्हर सामान्यतः खालील फंक्शन्स प्रदान करतो:
- **search_listings** - स्थान आणि निकषांनुसार Airbnb मालमत्तांचा शोध घ्या
- **get_listing_details** - विशिष्ट मालमत्तांबद्दल तपशीलवार माहिती मिळवा
- **check_availability** - विशिष्ट तारखांसाठी उपलब्धता तपासा
- **get_reviews** - मालमत्तांसाठी पुनरावलोकने मिळवा
- **get_host_info** - मालमत्तांच्या होस्टबद्दल माहिती मिळवा

### पूर्वअट:

- तुमच्या प्रणालीवर **Node.js** स्थापित असणे आवश्यक आहे
- MCP सर्व्हर पॅकेज डाउनलोड करण्यासाठी **इंटरनेट कनेक्शन**
- **NPX** उपलब्ध असणे (Node.js सोबत येते)

### कनेक्शनची चाचणी करणे:

तुम्ही MCP सर्व्हरला मॅन्युअली चालवून चाचणी करू शकता:
```bash
npx -y @openbnb/mcp-server-airbnb
```

हे OpenBnB MCP सर्व्हर डाउनलोड आणि सुरू करेल, ज्याला Semantic Kernel नंतर वास्तविक Airbnb डेटासाठी जोडतो.


## OpenBnB MCP सर्व्हरसोबत एजंट चालवणे

आता आपण असा AI एजंट चालवणार आहोत जो OpenBnB MCP सर्व्हरशी जोडतो आणि स्टॉकहोममधील 2 प्रौढ आणि 1 मुलासाठी खऱ्या Airbnb निवासस्थानांचा शोध घेतो. शोध निकष बदलण्यासाठी `user_inputs` यादीत बदल करण्यास मोकळे रहा.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

सारांश  
अभिनंदन! तुम्ही यशस्वीरित्या एक AI एजंट तयार केला आहे जो मॉडेल कॉन्टेक्स्ट प्रोटोकॉल (MCP) चा वापर करून वास्तविक जगातील निवासस्थान शोधाशी एकत्रित होतो:

वापरलेल्या तंत्रज्ञानाचा आढावा:  
- सेमॅंटिक कर्नल - Azure OpenAI सह बुद्धिमान एजंट तयार करण्यासाठी  
- Azure AI Foundry - LLM क्षमता आणि चॅट पूर्णतेसाठी  
- MCP (मॉडेल कॉन्टेक्स्ट प्रोटोकॉल) - मानकीकृत साधन एकत्रीकरणासाठी  
- OpenBnB MCP सर्व्हर - वास्तविक Airbnb शोध कार्यक्षमतेसाठी  
- Node.js/NPX - बाह्य MCP सर्व्हर चालवण्यासाठी  

तुम्ही काय शिकलात:  
- MCP एकत्रीकरण: सेमॅंटिक कर्नल एजंट्सना बाह्य MCP सर्व्हर्सशी जोडणे  
- रिअल-टाइम डेटा प्रवेश: थेट API च्या माध्यमातून वास्तविक Airbnb मालमत्तांचा शोध घेणे  
- प्रोटोकॉल संवाद: एजंट आणि MCP सर्व्हर दरम्यान stdio संवादाचा वापर  
- फंक्शन डिस्कव्हरी: MCP सर्व्हर्समधून उपलब्ध फंक्शन्स आपोआप शोधणे  
- स्ट्रीमिंग प्रतिसाद: फंक्शन कॉल्स रिअल-टाइममध्ये कॅप्चर करणे आणि लॉग करणे  
- HTML रेंडरिंग: एजंट प्रतिसादांना स्टाइल केलेल्या टेबल्स आणि इंटरॅक्टिव्ह डिस्प्लेने फॉरमॅट करणे  

पुढील पावले:  
- अतिरिक्त MCP सर्व्हर्स (हवामान, फ्लाइट्स, रेस्टॉरंट्स) समाकलित करा  
- MCP आणि A2A प्रोटोकॉल्स एकत्र करून मल्टी-एजंट सिस्टम तयार करा  
- तुमच्या स्वतःच्या डेटा स्रोतांसाठी सानुकूल MCP सर्व्हर्स तयार करा  
- सत्रांदरम्यान सतत संभाषणाची स्मृती अंमलात आणा  
- MCP सर्व्हर ऑर्केस्ट्रेशनसह एजंटला Azure Functions वर तैनात करा  
- वापरकर्ता प्रमाणीकरण आणि बुकिंग क्षमता जोडा  

MCP आर्किटेक्चरचे मुख्य फायदे:  
- मानकीकरण: AI एजंट्सना बाह्य साधनांशी जोडण्यासाठी सार्वत्रिक प्रोटोकॉल  
- रिअल-टाइम डेटा: विविध सेवांमधून थेट, अद्ययावत माहितीचा प्रवेश  
- विस्तारक्षमता: नवीन डेटा स्रोत आणि साधनांचे सहज एकत्रीकरण  
- परस्परसंवाद: विविध AI फ्रेमवर्क्स आणि एजंट प्लॅटफॉर्म्समध्ये कार्यक्षम  
- जबाबदाऱ्यांचे विभाजन: AI लॉजिक आणि बाह्य डेटा प्रवेश यामध्ये स्पष्ट भेद  



---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी कृपया लक्षात ठेवा की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून उद्भवलेल्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
